## Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt 
from scipy.stats import norm
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
t = yf.Ticker('TSLA')

In [4]:
# Get historical market data
hist = t.history(period="3mo")

In [6]:
# latest closing price and store it in a DataFrame
market_price = hist['Close'].iloc[-1]  # Extracting the closing price of the last record

In [5]:
eps = t.info['trailingEps']  # Extract trailing EPS, can be replaced with 'forwardEps' if preferred

In [8]:
# Fetching financial statements
financials = t.financials  # Fetch the income statement data
financials

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Tax Effect Of Unusual Items,0.0,-14080000.0,2970000.0,0.0,NaN
Tax Rate For Calcs,0.21,0.08,0.11,0.25,NaN
Normalized EBITDA,14796000000.0,17833000000.0,9598000000.0,4224000000.0,NaN
Total Unusual Items,0.0,-176000000.0,27000000.0,0.0,NaN
Total Unusual Items Excluding Goodwill,0.0,-176000000.0,27000000.0,0.0,NaN
Net Income From Continuing Operation Net Minority Interest,14999000000.0,12583000000.0,5524000000.0,721000000.0,NaN
Reconciled Depreciation,4667000000.0,3747000000.0,2911000000.0,2322000000.0,NaN
Reconciled Cost Of Revenue,79113000000.0,60609000000.0,40217000000.0,24906000000.0,NaN
EBITDA,14796000000.0,17657000000.0,9625000000.0,4224000000.0,NaN
EBIT,10129000000.0,13910000000.0,6714000000.0,1902000000.0,NaN


In [10]:
balance_sheet = t.balance_sheet  # Fetching the balance sheet data
balance_sheet

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Treasury Shares Number,0.0,NaN,NaN,NaN,NaN
Ordinary Shares Number,3185000000.0,3164000000.0,3099000000.0,2880000000.0,NaN
Share Issued,3185000000.0,3164000000.0,3099000000.0,2880000000.0,NaN
Net Debt,NaN,NaN,NaN,NaN,5533000000.0
Total Debt,9573000000.0,5748000000.0,8873000000.0,13279000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,29094000000.0,22185000000.0,17707000000.0,19384000000.0,NaN
Other Short Term Investments,12696000000.0,5932000000.0,131000000.0,0.0,NaN
Cash And Cash Equivalents,16398000000.0,16253000000.0,17576000000.0,19384000000.0,NaN
Cash Equivalents,495000000.0,2288000000.0,9548000000.0,NaN,NaN


In [12]:
# Extracting key financial figures for ratio calculations
net_income = financials.loc['Net Income'].iloc[0]  # Extract net income
net_income

14999000000.0

In [13]:
total_assets = balance_sheet.loc['Total Assets'].iloc[0]  # Extract total assets
total_assets

106618000000.0

In [16]:
liab_columns = [col for col in balance_sheet.index if 'liabilities' in col.lower()]
liab_columns #finding total liabilities in balance sheet

['Total Liabilities Net Minority Interest',
 'Total Non Current Liabilities Net Minority Interest',
 'Other Non Current Liabilities',
 'Non Current Deferred Liabilities',
 'Non Current Deferred Taxes Liabilities',
 'Current Liabilities',
 'Other Current Liabilities',
 'Current Deferred Liabilities']

In [17]:
# Get total liabilities and add it to the DF
# Calculating Total Liabilities by summing Current and Non-Current Liabilities
current_liabilities = balance_sheet.loc['Current Liabilities'].iloc[0]
non_current_liabilities = balance_sheet.loc['Total Non Current Liabilities Net Minority Interest'].iloc[0]


In [18]:
total_liabilities = current_liabilities + non_current_liabilities
total_liabilities


43009000000.0

In [19]:
shareholders_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]  # Extract shareholders' equity
shareholders_equity

62634000000.0

### Quantitative Analysis - Calculating Financial Ratios

In [20]:
# Calculating the Price-to-Earnings (P/E) Ratio
pe_ratio = market_price / eps
pe_ratio

np.float64(55.29089748692446)

In [21]:
# Calculating the Return on Assets (ROA)
roa = net_income / total_assets
roa

0.14067981016338704

In [23]:
# Calculating the Debt-to-Equity (D/E) Ratio
de_ratio = total_liabilities / shareholders_equity
de_ratio

0.6866717757128716

In [25]:
# financial ratios in a DataFrame
ratios = pd.DataFrame({
    'P/E Ratio': [pe_ratio],
    'ROA': [roa],
    'D/E Ratio': [de_ratio]
})

In [26]:
print("Financial Ratios:")
print(ratios)

Financial Ratios:
   P/E Ratio      ROA  D/E Ratio
0  55.290897  0.14068   0.686672


### Qualitative Analysis

In [34]:
# Setting up qualitative analysis aspects: Business Model, Competitive Advantage, Management and Governance
business_model = """Tesla Inc. designs, develops, manufactures, and sells electric vehicles and energy generation and storage systems in the United States, China, and internationally."""
business_model

'Tesla Inc. designs, develops, manufactures, and sells electric vehicles and energy generation and storage systems in the United States, China, and internationally.'

In [33]:
competitive_advantage = """Tesla's main competitive advantages include its strong brand recognition, innovative technology, and substantial investment in research and development."""
competitive_advantage

"Tesla's main competitive advantages include its strong brand recognition, innovative technology, and substantial investment in research and development."

In [32]:
management_governance = """Led by CEO Elon Musk, Tesla's management is known for its visionary approach and aggressive expansion strategies. Corporate governance is marked by a focus on sustainability and long-term market leadership."""
management_governance

"Led by CEO Elon Musk, Tesla's management is known for its visionary approach and aggressive expansion strategies. Corporate governance is marked by a focus on sustainability and long-term market leadership."

In [36]:
# summaries of qualitative analysis
print("Business Model:\n", business_model)

print("Competitive Advantage:\n", competitive_advantage)

print("Management and Governance:\n", management_governance)

Business Model:
 Tesla Inc. designs, develops, manufactures, and sells electric vehicles and energy generation and storage systems in the United States, China, and internationally.
Competitive Advantage:
 Tesla's main competitive advantages include its strong brand recognition, innovative technology, and substantial investment in research and development.
Management and Governance:
 Led by CEO Elon Musk, Tesla's management is known for its visionary approach and aggressive expansion strategies. Corporate governance is marked by a focus on sustainability and long-term market leadership.


Thank you.